## Prerequisites

In [56]:
from main import initialized_lark_from_file

lark = initialized_lark_from_file('../grammar.lark')

def print_parsed(text_to_parse: str):
    print(lark.parse(text_to_parse).pretty())

## Import statements

Grammar

```txt
start: import_statement* ((statement | declaration) (_NL (statement | declaration))*)?

import_statement: import_name | import_from

import_name: IMPORT as_name ("," as_name)*

import_from: FROM (RELATIVE_LOCATION)? (NAME ((".") NAME)*)? IMPORT import_targets

?import_targets: as_name ("," as_name)* | IMPORT_ALL

?as_name: NAME ("as" NAME)?
```

### Test '*' operator

In [2]:
print(lark.parse(r"import a import b from c import d").pretty())

start
  import_statement
    import_name
      import
      a
  import_statement
    import_name
      import
      b
  import_statement
    import_from
      from
      c
      import
      d



### Test multiple as imports

Expected: from ..car.mercedes import 2 targets

In [3]:
print(lark.parse(r"""
from ..car.mercedes import print_car as p_cr, lol_kek as l_k
""").pretty())

start
  import_statement
    import_from
      from
      ..
      car
      mercedes
      import
      import_targets
        as_name
          print_car
          p_cr
        as_name
          lol_kek
          l_k



### Test '.' and '*' symbols in import statements

Expected: correct recognition of symbols

In [4]:
print(lark.parse(r"""
from . import *
""").pretty())

start
  import_statement
    import_from
      from
      .
      import
      *



## Sequence of statements and declarations

Grammar

```txt
start: import_statement* ((statement | declaration) (_NL (statement | declaration))*)?
```

### Test a sequence of statements without a new line

Expected: throw an error because y should go after a new line

In [5]:
try:
    test_snippet = r"x = foo() y = bar"
    lark.parse(test_snippet).pretty()
except Exception as ex:
    print("OK, exception was raised")
    print(ex)
else:
    raise Exception("Exception must be raised")

OK, exception was raised
No terminal defined for 'y' at line 1 col 11

x = foo() y = bar
          ^

Expecting: {'COMPARISON_OPERATOR', '_NL', 'LESSTHAN', 'ADDITIVE_OPERATOR', '__ANON_0', 'DOT', 'LSQB', 'LPAR', '__ANON_1', 'MULTIPLICATIVE_OPERATOR', 'EQUALITY_OPERATOR'}



### Test a sequence of statements and declarations

Expected: print 3 statements

In [6]:
test_snippet = r"""
x int val = 2
prt(some_val int) str {
    return "prt" + str(some_val)
}
prt(x)
"""
print(lark.parse(test_snippet).pretty())

start
  statement
    assignment
      directly_assignable_expression
        variable_declaration
          x
          type
            simple_user_type	int
          val
      disjunction
        conjunction
          equality
            comparison
              additive_expression
                multiplicative_expression
                  prefix_unary_expression
                    postfix_unary_expression
                      primary_expression
                        simple_literal	2
  declaration
    function_declaration
      prt
      function_parameters
        some_val
        type
          simple_user_type	int
      str
      statements_block
        statement
          jump_statement
            return
            disjunction
              conjunction
                equality
                  comparison
                    additive_expression
                      multiplicative_expression
                        prefix_unary_expression
                          postf

## Test function declaration

### Grammar

```txt
function_declaration: NAME type_arguments? function_parameters
FUNCTION_VISIBILITY_MODIFIER? (ABSTRACT|OVERRIDDEN)? FUNCTION_RETURN_TYPE? "{" statements_block "}"
```

### Function declaration with type parameters

Expected: one function declaration with 2 type parameters

In [7]:
print_parsed(r"""
print<T1, T2>(){
    return "With type"
}""")

start
  declaration
    function_declaration
      print
      type_arguments
        type
          simple_user_type	T1
        type
          simple_user_type	T2
      function_parameters
      statements_block
        statement
          jump_statement
            return
            disjunction
              conjunction
                equality
                  comparison
                    additive_expression
                      multiplicative_expression
                        prefix_unary_expression
                          postfix_unary_expression
                            primary_expression
                              simple_literal	"With type"



### Function declaration without type parameters

Expected: just 1 function declaration

In [8]:
print_parsed(r"""
print(){
    return "Without type"
}"""
)

start
  declaration
    function_declaration
      print
      function_parameters
      statements_block
        statement
          jump_statement
            return
            disjunction
              conjunction
                equality
                  comparison
                    additive_expression
                      multiplicative_expression
                        prefix_unary_expression
                          postfix_unary_expression
                            primary_expression
                              simple_literal	"Without type"



### Function declaration with parameters

Expected: 1 function declaration with parameters and their types

In [9]:
print_parsed(r"""
print(arg1 int, arg2 str){
    return str(arg1) + arg2
}"""
)

start
  declaration
    function_declaration
      print
      function_parameters
        arg1
        type
          simple_user_type	int
        arg2
        type
          simple_user_type	str
      statements_block
        statement
          jump_statement
            return
            disjunction
              conjunction
                equality
                  comparison
                    additive_expression
                      multiplicative_expression
                        prefix_unary_expression
                          postfix_unary_expression
                            primary_expression	str
                            postfix_unary_suffix
                              call_suffix
                                function_call_arguments
                                  disjunction
                                    conjunction
                                      equality
                                        comparison
                                     

### Function declaration with private modifier and type parametes

In [10]:
print_parsed(r"""
print<T1, T2>() private {
    return "private"
}"""
)

start
  declaration
    function_declaration
      print
      type_arguments
        type
          simple_user_type	T1
        type
          simple_user_type	T2
      function_parameters
      private
      statements_block
        statement
          jump_statement
            return
            disjunction
              conjunction
                equality
                  comparison
                    additive_expression
                      multiplicative_expression
                        prefix_unary_expression
                          postfix_unary_expression
                            primary_expression
                              simple_literal	"private"



### Function declaration with *overridden* hint

In [11]:
print_parsed(r"""
print() private overridden {
    return "overridden"
}"""
)

start
  declaration
    function_declaration
      print
      function_parameters
      private
      overridden
      statements_block
        statement
          jump_statement
            return
            disjunction
              conjunction
                equality
                  comparison
                    additive_expression
                      multiplicative_expression
                        prefix_unary_expression
                          postfix_unary_expression
                            primary_expression
                              simple_literal	"overridden"



### Function declaration with a return type provided

In [12]:
print_parsed(r"""
print() private ReturnType {
    return new("arg for the constructor")
}""")

start
  declaration
    function_declaration
      print
      function_parameters
      private
      ReturnType
      statements_block
        statement
          jump_statement
            return
            disjunction
              conjunction
                equality
                  comparison
                    additive_expression
                      multiplicative_expression
                        prefix_unary_expression
                          postfix_unary_expression
                            primary_expression	new
                            postfix_unary_suffix
                              call_suffix
                                function_call_arguments
                                  disjunction
                                    conjunction
                                      equality
                                        comparison
                                          additive_expression
                                            multiplicative

## Loops

### Grammar

```txt
for_statement: label? "for" NAME "in" expression "{" statements_block "}"
while_statement: label? "while" expression "{" statements_block "}"
```

### A for loop with a label and break statement

Expected: proper recognition of a label and jump statement inside the for_statement

In [13]:
print_parsed(r"""
myLabel@
for x in db.values {
    print(x)
    break@ myLabel
}""")

start
  statement
    for_statement
      label	myLabel
      x
      disjunction
        conjunction
          equality
            comparison
              additive_expression
                multiplicative_expression
                  prefix_unary_expression
                    postfix_unary_expression
                      primary_expression	db
                      postfix_unary_suffix
                        navigation_suffix	values
      statements_block
        statement
          disjunction
            conjunction
              equality
                comparison
                  additive_expression
                    multiplicative_expression
                      prefix_unary_expression
                        postfix_unary_expression
                          primary_expression	print
                          postfix_unary_suffix
                            call_suffix
                              function_call_arguments
                                disjunction
     

### A for loop with a nested loop

In [14]:
print_parsed(r"""
for x in db.values {
    print(x)
    print("outer")
    for y in db.otherValues {
        print(y)
        print("inner")
    }
}""")

start
  statement
    for_statement
      x
      disjunction
        conjunction
          equality
            comparison
              additive_expression
                multiplicative_expression
                  prefix_unary_expression
                    postfix_unary_expression
                      primary_expression	db
                      postfix_unary_suffix
                        navigation_suffix	values
      statements_block
        statement
          disjunction
            conjunction
              equality
                comparison
                  additive_expression
                    multiplicative_expression
                      prefix_unary_expression
                        postfix_unary_expression
                          primary_expression	print
                          postfix_unary_suffix
                            call_suffix
                              function_call_arguments
                                disjunction
                         

### A for loop with "in" omitted

In [15]:
try:
    print_parsed(r"""
for x db.values {
    print(x)
}""")
except Exception as ex:
    print("OK, exception was raised")
    print(ex)
else:
    raise Exception("Exception had must be raised here")

OK, exception was thrown
No terminal defined for 'd' at line 2 col 7

for x db.values {
      ^

Expecting: {'IN', 'LESSTHAN', 'DOT', 'VAL', 'EQUAL', 'VAR'}



### Test while statement

Expected: proper recognition of if expression in the while statement

In [16]:
print_parsed(r"""
while if x < 5 {closeToZero(x)} else {true} {
    x -= 1
}"""
)

start
  statement
    while_statement
      disjunction
        conjunction
          equality
            comparison
              additive_expression
                multiplicative_expression
                  prefix_unary_expression
                    postfix_unary_expression
                      primary_expression
                        if_expression
                          disjunction
                            conjunction
                              equality
                                comparison
                                  additive_expression
                                    multiplicative_expression
                                      prefix_unary_expression
                                        postfix_unary_expression
                                          primary_expression	x
                                  <
                                  additive_expression
                                    multiplicative_expression
                      

## Operators precedence

| Precedence | Title          | Symbols               |
|------------|----------------|-----------------------|
| Highest    | Postfix        | .                     |
| ^          | Prefix         | -, +, !               |
| \|         | Multiplicative | *, /, %               |
| \|         | Additive       | +, -                  |
| \|         | Comparison     | <, >, <=, >=          |
| \|         | Equality       | ==, !=                |
| \|         | Conjunction    | &&                    |
| ^          | Disjunction    | \|\|                  |
| Lowest     | Assignment     | =, +=, -=, *=, /=, %= |

### Postfix and prefix and call suffix

Expected: prefixExpr -> postfixExpr with 2 suffixes (navigation and call)

In [17]:
print_parsed(r"""+person.sayHello()""")

start
  statement
    disjunction
      conjunction
        equality
          comparison
            additive_expression
              multiplicative_expression
                prefix_unary_expression
                  +
                  postfix_unary_expression
                    primary_expression	person
                    postfix_unary_suffix
                      navigation_suffix	sayHello
                    postfix_unary_suffix
                      call_suffix
                        function_call_arguments



### Postfix and equality and suffix

Expected: comparison first then id with suffixes

In [18]:
print_parsed(r""""OK" == person.mood().asString()""")

start
  statement
    disjunction
      conjunction
        equality
          comparison
            additive_expression
              multiplicative_expression
                prefix_unary_expression
                  postfix_unary_expression
                    primary_expression
                      simple_literal	"OK"
          ==
          comparison
            additive_expression
              multiplicative_expression
                prefix_unary_expression
                  postfix_unary_expression
                    primary_expression
                      identifier
                        person
                        mood
                    postfix_unary_suffix
                      call_suffix
                        function_call_arguments
                    postfix_unary_suffix
                      navigation_suffix	asString
                    postfix_unary_suffix
                      call_suffix
                        function_call_arguments



### Conjunction and disjunction

Expected: disjunction -> 2 conjunctions

In [19]:
print_parsed(r"""
procrastinateToday() && watchMovies() ||
procrastinateTomorrow() && workHard()
""")

start
  statement
    disjunction
      conjunction
        equality
          comparison
            additive_expression
              multiplicative_expression
                prefix_unary_expression
                  postfix_unary_expression
                    primary_expression	procrastinateToday
                    postfix_unary_suffix
                      call_suffix
                        function_call_arguments
        equality
          comparison
            additive_expression
              multiplicative_expression
                prefix_unary_expression
                  postfix_unary_expression
                    primary_expression	watchMovies
                    postfix_unary_suffix
                      call_suffix
                        function_call_arguments
      conjunction
        equality
          comparison
            additive_expression
              multiplicative_expression
                prefix_unary_expression
                  postfix_unary_express

### Assignment and comparison and disjunction and parentheses

Expected: assignment -> conjunction -> disjunction

In [20]:
print_parsed(r"""
x int val = leftTasksNumber < 5 && (shouldWorkToday() || "Work tomorrow")
""")

start
  statement
    assignment
      directly_assignable_expression
        variable_declaration
          x
          type
            simple_user_type	int
          val
      disjunction
        conjunction
          equality
            comparison
              additive_expression
                multiplicative_expression
                  prefix_unary_expression
                    postfix_unary_expression
                      primary_expression	leftTasksNumber
              <
              additive_expression
                multiplicative_expression
                  prefix_unary_expression
                    postfix_unary_expression
                      primary_expression
                        simple_literal	5
          equality
            comparison
              additive_expression
                multiplicative_expression
                  prefix_unary_expression
                    postfix_unary_expression
                      primary_expression
                    

### Prefix and additive

Expected: proper recognition of prefix operators

1. assignment -> additive expression and 2 prefix operators
2. assignment -> 1 prefix operator

In [33]:
print_parsed(r"""
x bool val = a +-!b
c += !d
""")

start
  statement
    assignment
      directly_assignable_expression
        variable_declaration
          x
          type
            simple_user_type	bool
          val
      disjunction
        conjunction
          equality
            comparison
              additive_expression
                multiplicative_expression
                  prefix_unary_expression
                    postfix_unary_expression
                      primary_expression	a
                +
                multiplicative_expression
                  prefix_unary_expression
                    -
                    !
                    postfix_unary_expression
                      primary_expression	b
  statement
    assignment
      assignable_expression
        prefix_unary_expression
          postfix_unary_expression
            primary_expression	c
      +=
      disjunction
        conjunction
          equality
            comparison
              additive_expression
                multiplicati

## Interfaces

### Grammar

```txt
interface_declaration: NAME "interface" type_arguments? parent_interfaces "{" interface_body "}"

parent_interfaces: (NAME ("," NAME)*)?

interface_body: interface_function_declaration+

interface_function_declaration: (function_declaration | function_declaration_without_body)

function_declaration_without_body: NAME type_arguments? function_parameters FUNCTION_RETURN_TYPE
```

### Interface only with function declarations without body

Expected a Male interface with 2 type parameters, 3 parent interfaces and 3 function_declarations without a body

In [34]:
print_parsed(r"""
Male interface <T1, T2> Human, Printable, Hashable {
    hashCode() int
    printable<T1>() void
    shouldSleepAtTime(time Time) bool
}
"""
)

start
  declaration
    interface_declaration
      Male
      type_arguments
        type
          simple_user_type	T1
        type
          simple_user_type	T2
      interface_parents
        Human
        Printable
        Hashable
      interface_body
        interface_function_declaration
          function_declaration_without_body
            hashCode
            function_parameters
            int
        interface_function_declaration
          function_declaration_without_body
            printable
            type_arguments
              type
                simple_user_type	T1
            function_parameters
            void
        interface_function_declaration
          function_declaration_without_body
            shouldSleepAtTime
            function_parameters
              time
              type
                simple_user_type	Time
            bool



### Test if it will be ok to make multiple declarations of functions without body without a new line

In [35]:
print_parsed(r"""
Male interface <T1, T2> Human, Printable, Hashable {
    hashCode() int printable<T1>() void shouldSleepAtTime(time Time) bool
}
"""
)

start
  declaration
    interface_declaration
      Male
      type_arguments
        type
          simple_user_type	T1
        type
          simple_user_type	T2
      interface_parents
        Human
        Printable
        Hashable
      interface_body
        interface_function_declaration
          function_declaration_without_body
            hashCode
            function_parameters
            int
        interface_function_declaration
          function_declaration_without_body
            printable
            type_arguments
              type
                simple_user_type	T1
            function_parameters
            void
        interface_function_declaration
          function_declaration_without_body
            shouldSleepAtTime
            function_parameters
              time
              type
                simple_user_type	Time
            bool



### Test if it will be ok to make multiple declarations of functions without AND WITH body without a new line

Expected: 5 correct function declarations

In [37]:
print_parsed(r"""
Male interface <T1, T2> Human, Printable, Hashable {
    hashCode() int printable<T1>() void mood() str {
    return "OK"
    } shouldSleepAtTime(time Time) bool wantsToEat() bool {return true}
}
"""
)

UnexpectedCharacters: No terminal defined for '<' at line 3 col 25

    hashCode() printable<T1>() void mood() str {
                        ^

Expecting: {'NAME', 'RBRACE', 'LBRACE'}


### Test if a return type will be missed

Expected: raised exception

In [39]:
try:
    print_parsed(r"""
    Male interface <T1, T2> Human, Printable, Hashable {
        # printable has a missed type here type is missed
        hashCode() int printable<T1>() mood() str { return "OK" }
    }
    """
    )
except Exception as ex:
    print("OK, exception was raised")
    print(ex)
else:
    raise Exception("Exception must be raised")

KeyboardInterrupt: 

## Classes

### Grammar

```txt
class_declaration: NAME "class" type_arguments? ABSTRACT? class_hint "{" class_body "}"
// parent class and interfaces (parent class must stay first)
class_hint: (NAME ("," NAME)*)?
class_body: (class_member_declaration)*
class_member_declaration: function_declaration | property_declaration | abstract_function_declaration
```

### Abstract class declaration

Expected: abstract class declaration with 1 function declared

In [46]:
print_parsed(r"""
Male class abstract Human, Printable {
    foo() abstract str
}
""")

start
  declaration
    class_declaration
      Male
      abstract
      class_hint
        Human
        Printable
      class_body
        class_member_declaration
          abstract_function_declaration
            foo
            function_parameters
            abstract
            str



### Multiple function declarations

Expected: abstract class declaration with 4 functions

In [47]:
print_parsed(r"""
Male class abstract {
    foo() abstract str
    bar() {
        return "bar"
    }
    kek() abstract bool
    lol() int {
        return 10
    }
}""")


start
  declaration
    class_declaration
      Male
      abstract
      class_hint
      class_body
        class_member_declaration
          abstract_function_declaration
            foo
            function_parameters
            abstract
            str
        class_member_declaration
          function_declaration
            bar
            function_parameters
            statements_block
              statement
                jump_statement
                  return
                  disjunction
                    conjunction
                      equality
                        comparison
                          additive_expression
                            multiplicative_expression
                              prefix_unary_expression
                                postfix_unary_expression
                                  primary_expression
                                    simple_literal	"bar"
        class_member_declaration
          abstract_function_declarati

### Class declarations and property declarations

Expected: 1 function -> 2 properties -> 1 function

In [52]:
print_parsed(r"""
Male class abstract {
    foo() str {return "foo"}
    bar int val
    lol bool public var = false
    kek() abstract bool
}
""")

start
  declaration
    class_declaration
      Male
      abstract
      class_hint
      class_body
        class_member_declaration
          function_declaration
            foo
            function_parameters
            str
            statements_block
              statement
                jump_statement
                  return
                  disjunction
                    conjunction
                      equality
                        comparison
                          additive_expression
                            multiplicative_expression
                              prefix_unary_expression
                                postfix_unary_expression
                                  primary_expression
                                    simple_literal	"foo"
        class_member_declaration
          property_declaration	bar
        class_member_declaration
          property_declaration
            int
            val
        class_member_declaration
          prope

### Declarations without a new line

Expected: throw an error

In [58]:
try:
    print_parsed(r"""
    Male class abstract {
        foo() str {return "foo"} bar int val lol bool public var = false kek() abstract bool
    }
    """)
except Exception as ex:
    print("OK, it raises an exception")
    print(ex)
else:
    raise Exception("It must raise an exception")

OK, it raises an exception
No terminal defined for 'b' at line 3 col 34

        foo() str {return "foo"} bar int val lol bool public var = false 
                                 ^

Expecting: {'RBRACE', '_NL'}

